In [1]:
import pandas as pd

In [2]:
# 读取excel数据

xiaoshou = pd.read_excel('data/17xiaoshou.xls',header=1)
tuihuo = pd.read_excel('data/17tuihuo.xls',header=1)
zengjiu = pd.read_excel('data/17zengjiu.xls',header=1)

In [3]:
# 转换日期列

xiaoshou['日期'] = pd.to_datetime(xiaoshou['日期'])
xiaoshou['日期'] = xiaoshou['日期'].dt.date

tuihuo['日期'] = pd.to_datetime(tuihuo['日期'])
tuihuo['日期'] = tuihuo['日期'].dt.date

zengjiu['日期'] = pd.to_datetime(zengjiu['日期'])
zengjiu['日期'] = zengjiu['日期'].dt.date

In [4]:
# 整理“tuihuo”数据

# 重命名列名
tuihuo.rename(columns = {'单价': '退货单价', '数量': '退货数量', '金额': '退货金额', '去向': '退货去向'}, inplace=True)

# 添加一列：提取非换货的退货数量
tuihuo['净退货数量'] = tuihuo['退货数量']*(tuihuo['换货'] != '是')

In [5]:
# 整理“zengjiu”数据

# 重命名列名
zengjiu.rename(columns = {'成本': '赠酒成本', '数量': '赠酒数量', '价值': '赠酒价值', '缘由': '赠酒缘由', '客户': '客户名称'}, inplace=True)

# 添加一列：提取还预存数
zengjiu['还预存数'] = zengjiu['赠酒数量']*(zengjiu['对抵'] == '是')

In [6]:
# 合并三个表格，生成最终的数据表

# 使用concat合并
# http://pandas.pydata.org/pandas-docs/stable/merging.html
df0 = pd.concat([xiaoshou,tuihuo,zengjiu],ignore_index=True)

# 按照日期进行降序排列
df1 = df0.sort_values(['日期'],ascending=False)

# 将所有的NaN值处理成0
df2 = df1.fillna(0)

# 添加一列：[销售数量]-[赠酒数量]+[还预存数]-[净退货数量]
df2['结算数量'] = df2['销售数量']-df2['赠酒数量']+df2['还预存数']-df2['净退货数量']

# 添加一列：[赠酒数量]-[还预存数]
df2['净赠酒数量'] = df2['赠酒数量']-df2['还预存数']

# 添加一列：[销售数量]-[净退货数量]
df2['净出库数量']= df2['销售数量']-df2['净退货数量']

# 对列进行最终排序

df2=df2[['销售员', '产品名', '产品类别', '二级类别', '客户名称','客户类别', '客户分级', '销售数量', '赠送数量', '异价数量','正常数量', '正常单价', '异价单价', '异价金额', '常价金额','合计金额','备注','日期','赠酒成本','赠酒数量','赠酒价值','对抵','赠酒缘由','还预存数','退货单价','退货数量','退货金额','换货','退货去向','转给人','净退货数量','结算数量','净赠酒数量','净出库数量']]

In [7]:
df2.to_excel('test.xlsx',index=False)